In [1]:
import sys
import os

def setup_workdirectory(
    repo_url="https://github.com/kikaymusic/EscuderoRodriguezSanchez.git",
    branch="dev",
    repo_name="EscuderoRodriguezSanchez",
    directoy_name="Entornos_Complejos"
):
    """
    Función para detectar si el código se está ejecutando en Google Colab o en local
    y configurar el entorno de trabajo en consecuencia.
    """
    # Detectamos si estamos en Google Colab
    in_colab = "google.colab" in sys.modules

    if in_colab:
        print("Entorno detectado: Google Colab")

        # Definimos la ruta del repositorio en Colab
        repo_path = f"/content/{repo_name}"

        # Borramos el repositorio si ya existe para evitar conflictos
        if os.path.exists(repo_path):
            os.system(f"rm -rf {repo_path}")

        # Clonamos la rama especificada del repositorio
        os.system(f"git clone -b {branch} --single-branch {repo_url}")

        # Cambiamos al directorio del repositorio
        os.system(f"%cd {repo_path}")

        # Añadimos el directorio especificado al path
        sys.path.append(f'/content/{repo_name}/{directoy_name}')

    else:
        print("Entorno detectado: Local")

        # Añadir el directorio especificado al path de Python
        notebook_dir = os.path.dirname(os.path.abspath('__file__')) if '__file__' in globals() else os.getcwd()
        k_brazos_dir = notebook_dir if directoy_name in notebook_dir else os.path.join(notebook_dir, directoy_name)
        if k_brazos_dir not in sys.path:
            sys.path.insert(0, k_brazos_dir)


setup_workdirectory()

Entorno detectado: Local


In [2]:
import gymnasium as gym

## Entorno Blackjack para tipo Tabular (Espacio de estados discreto) off-policy

En Off-policy, la Política de Comportamiento es distinta a la Política Objetivo. El agente genera experiencia moviéndose por el entorno de una forma (explorando mucho), pero actualiza sus valores asumiendo que está siguiendo una política completamente diferente (generalmente, una política 100% óptima o Greedy).

En On-policy, la Política de Comportamiento es exactamente la misma que la Política Objetivo. El agente aprende el valor de la política que está utilizando actualmente para tomar decisiones.

In [3]:
from Entornos_Complejos.src.agents.montecarlo import AgentMonteCarlo
from Entornos_Complejos.src.utils import SEMILLA
from Entornos_Complejos.src.policies.epsilon_greedy import EpsilonGreedyPolicy

# Siguiendo las reglas de Sutton & Barto
env = gym.make("Blackjack-v1", render_mode="rgb_array", sab=True)

# Inicializamos la política con la semilla
behavior_policy = EpsilonGreedyPolicy(epsilon=0.1, n_actions=env.action_space.n, seed=SEMILLA)  # Política epsilon-greedy para el entrenamiento
target_policy = EpsilonGreedyPolicy(epsilon=0.0, n_actions=env.action_space.n, seed=SEMILLA)  # Política full-greedy

# El agente usa las políticas
agent = AgentMonteCarlo(env, target_policy, behavior_policy)

# Al iniciar el entrenamiento, le pasamos la semilla al entorno
state, info = env.reset(seed=SEMILLA)